# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

print(tf)

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

<module 'tensorflow' from 'd:\\programdata\\anaconda3\\envs\\tensorflow-avx\\lib\\site-packages\\tensorflow\\__init__.py'>


### Hyperparameters

In [2]:
### General parameters
max_steps = 5e5 # Set maximum number of steps to run environment.
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 2500 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "3dball" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 12 # Number of gradient descent steps per batch of experiences.
num_layers = 3 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 4096 # How large the experience buffer should be before gradient descent.
learning_rate = 1.5e-4 # Model learning rate.
hidden_units = 80 # Number of units in hidden layer.
batch_size = 1024 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'Ball3DAcademy' started successfully!


Unity Academy name: Ball3DAcademy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: Ball3DBrain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 18
        Action space type: continuous
        Action space size (per agent): 6
        Memory space size (per agent): 0
        Action descriptions: rot z, rot x, rot y, move x, move y, move z


### Train the Agent(s)

In [4]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Step: 2500. Mean Reward: -0.5222308470265059. Std of Reward: 0.4651253859278168.
Step: 5000. Mean Reward: -0.09790442811376997. Std of Reward: 0.9741602204579446.
Step: 7500. Mean Reward: 0.6996831366959544. Std of Reward: 2.219883808666055.
Step: 10000. Mean Reward: 1.8355062384879675. Std of Reward: 3.7586725155475955.
Step: 12500. Mean Reward: 3.133852128480086. Std of Reward: 5.020025479647246.
Step: 15000. Mean Reward: 6.60581354016235. Std of Reward: 9.107321939188475.
Step: 17500. Mean Reward: 6.481737854255683. Std of Reward: 8.755170151376799.
Step: 20000. Mean Reward: 10.306838699883842. Std of Reward: 13.088978536853588.
Step: 22500. Mean Reward: 11.629050286699995. Std of Reward: 14.275091108002103.
Step: 25000. Mean Reward: 16.06578516498151. Std of Reward: 18.133609811254217.
Step: 27500. Mean Reward: 19.096468722906735. Std of Reward: 17.191318204471877.
Step: 30000. Mean Reward: 10.743319416717364. Std of Reward: 15.580513216447326.
Step: 32500. Mean Reward: 7.938519207

Step: 255000. Mean Reward: 42.757092022872634. Std of Reward: 22.525882094252115.
Step: 257500. Mean Reward: 24.602719821477336. Std of Reward: 23.462478036887514.
Step: 260000. Mean Reward: 31.103824573009728. Std of Reward: 25.233491881853155.
Step: 262500. Mean Reward: 24.585503683714904. Std of Reward: 23.874756780498203.
Step: 265000. Mean Reward: 24.536298507201867. Std of Reward: 22.91321139685587.
Step: 267500. Mean Reward: 26.925739320278364. Std of Reward: 24.24162650151922.
Step: 270000. Mean Reward: 38.04735637427107. Std of Reward: 27.720049653307434.
Step: 272500. Mean Reward: 39.46044291233467. Std of Reward: 22.727251046800482.
Step: 275000. Mean Reward: 27.5411726494002. Std of Reward: 24.97362692658726.
Step: 277500. Mean Reward: 22.575632517846547. Std of Reward: 23.952243913945484.
Step: 280000. Mean Reward: 27.489343817903855. Std of Reward: 23.177938306458262.
Step: 282500. Mean Reward: 34.95797619739082. Std of Reward: 25.309746041298215.
Step: 285000. Mean Rewar

INFO:tensorflow:Restoring parameters from ./models/ppo\model-500001.cptk


INFO:tensorflow:Froze 9 variables.


INFO:tensorflow:Froze 9 variables.


Converted 9 variables to const ops.


### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [ ]:
export_graph(model_path, env_name)